<a href="https://colab.research.google.com/github/AFBA1993/Projects/blob/main/TemporalSeries/PAGS/PAGS_LSTM_Func.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import io
from keras.metrics import mean_squared_error
from keras.models import load_model

In [5]:
def LSTM_PAGS(LOOK_BACK, DROPUT, NEURONS, EPOCHS, BATCH_SIZE ):

  # Loading PAGS raw database:
  url = ('https://raw.githubusercontent.com/AFBA1993/Projects/main/'+
       'TemporalSeries/PAGS/PAGSattempt.csv')
  url_data = requests.get(url).content
  raw_data = pd.read_csv(io.StringIO(url_data.decode('utf-8')))
  raw_data = raw_data[["Date","Close"]]
  raw_data.index = pd.to_datetime(raw_data["Date"])
  raw_data.drop("Date", axis = 1, inplace =  True)

  #Spliting data into train and test sections:
  SPLIT_PERCENT = 0.80
  split = int(SPLIT_PERCENT*len(raw_data))
  train = raw_data[:split]
  test = raw_data[split:]

  #Shaping Training Data:
  input_train = train.values
  input_train = input_train.reshape((-1, 1))
  scaler = MinMaxScaler(feature_range = (0,1))
  input_train = scaler.fit_transform(input_train)

   
  X_train = []
  y_train = []

  for i in range (LOOK_BACK, len(input_train)):
    X_train.append(input_train[i-LOOK_BACK:i,0])
    y_train.append(input_train[i,0])

  X_train, y_train = np.array(X_train), np.array(y_train)
  X_train = np.reshape(X_train, (X_train.shape[0], LOOK_BACK, 1))

  #Configuring LSTM:

  model = Sequential()
  model.add(LSTM(units = NEURONS, return_sequences = True,
                    input_shape = (X_train.shape[1], 1)))
  model.add(Dropout(DROPUT))

  model.add(LSTM(units = NEURONS, return_sequences = True))
  model.add(Dropout(DROPUT))

  model.add(LSTM(units = NEURONS, return_sequences = True))
  model.add(Dropout(DROPUT))

  model.add(LSTM(units = NEURONS))
  model.add(Dropout(DROPUT))


  model.add(Dense(units = 1, activation = 'linear'))
  model.compile(optimizer = 'rmsprop', loss = 'mse',
                    metrics = ['mse'])
  
  model.fit(X_train, y_train, epochs = EPOCHS, batch_size = BATCH_SIZE, 
                                                            workers=-1)

  #Testing LSTM in the train section:
  LSTM_train = model.predict(X_train)
  LSTM_train = scaler.inverse_transform(LSTM_train)
  LSTM_train = pd.DataFrame(LSTM_train, columns = ['Close'])
  LSTM_train["Date"] = train[LOOK_BACK:].index
  LSTM_train.index = pd.to_datetime(LSTM_train["Date"])
  LSTM_train.drop("Date", axis = 1, inplace =  True)

  train_mse = np.reshape(train.values, (train.values.shape[0]))
  LSTM_train_mse = np.reshape(LSTM_train.values, (LSTM_train.values.shape[0]))
  MSE_train = mean_squared_error(train_mse[LOOK_BACK:], LSTM_train_mse)
  
  #Testing LSTM in the train section:
  complete_data = raw_data.values
  input_test = complete_data[len(complete_data) - len(test) - LOOK_BACK:]
  input_test = input_test.reshape((-1, 1))
  input_test = scaler.transform(input_test)

  X_test = []
  for i in range(LOOK_BACK, len(input_test)):
    X_test.append(input_test[i-LOOK_BACK:i,0])
  X_test = np.array(X_test)
  X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

  LSTM_test = model.predict(X_test)
  LSTM_test = scaler.inverse_transform(LSTM_test)

  LSTM_test = pd.DataFrame(LSTM_test, columns = ['Close'])
  LSTM_test["Date"] = test.index
  LSTM_test.index = pd.to_datetime(LSTM_test["Date"])
  LSTM_test.drop("Date", axis = 1, inplace =  True) 

  test_mse = np.reshape(test.values, (test.values.shape[0]))
  LSTM_test_mse = np.reshape(LSTM_test.values, (LSTM_test.values.shape[0]))
  MSE_test = mean_squared_error(test_mse, LSTM_test_mse)

  return MSE_train, MSE_test, model

In [14]:
def LSTM_PAGS_VAL(LOOK_BACK, model):

  # Loading PAGS raw database:
  url = ('https://raw.githubusercontent.com/AFBA1993/Projects/main/'+
       'TemporalSeries/PAGS/PAGSattempt.csv')
  url_data = requests.get(url).content
  raw_data = pd.read_csv(io.StringIO(url_data.decode('utf-8')))
  raw_data = raw_data[["Date","Close"]]
  raw_data.index = pd.to_datetime(raw_data["Date"])
  raw_data.drop("Date", axis = 1, inplace =  True)

  #Spliting data into train and test sections:
  SPLIT_PERCENT = 0.80
  split = int(SPLIT_PERCENT*len(raw_data))
  train = raw_data[:split]
  test = raw_data[split:]

  #Shaping Training Data:
  input_train = train.values
  input_train = input_train.reshape((-1, 1))
  scaler = MinMaxScaler(feature_range = (0,1))
  input_train = scaler.fit_transform(input_train)

   
  X_train = []
  y_train = []

  for i in range (LOOK_BACK, len(input_train)):
    X_train.append(input_train[i-LOOK_BACK:i,0])
    y_train.append(input_train[i,0])

  X_train, y_train = np.array(X_train), np.array(y_train)
  X_train = np.reshape(X_train, (X_train.shape[0], LOOK_BACK, 1))

  #Testing LSTM in the train section:
  LSTM_train = model.predict(X_train)
  LSTM_train = scaler.inverse_transform(LSTM_train)
  LSTM_train = pd.DataFrame(LSTM_train, columns = ['Close'])
  LSTM_train["Date"] = train[LOOK_BACK:].index
  LSTM_train.index = pd.to_datetime(LSTM_train["Date"])
  LSTM_train.drop("Date", axis = 1, inplace =  True)

  train_mse = np.reshape(train.values, (train.values.shape[0]))
  LSTM_train_mse = np.reshape(LSTM_train.values, (LSTM_train.values.shape[0]))
  MSE_train = mean_squared_error(train_mse[LOOK_BACK:], LSTM_train_mse)
  
  #Testing LSTM in the train section:
  complete_data = raw_data.values
  input_test = complete_data[len(complete_data) - len(test) - LOOK_BACK:]
  input_test = input_test.reshape((-1, 1))
  input_test = scaler.transform(input_test)

  X_test = []
  for i in range(LOOK_BACK, len(input_test)):
    X_test.append(input_test[i-LOOK_BACK:i,0])
  X_test = np.array(X_test)
  X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

  LSTM_test = model.predict(X_test)
  LSTM_test = scaler.inverse_transform(LSTM_test)

  LSTM_test = pd.DataFrame(LSTM_test, columns = ['Close'])
  LSTM_test["Date"] = test.index
  LSTM_test.index = pd.to_datetime(LSTM_test["Date"])
  LSTM_test.drop("Date", axis = 1, inplace =  True) 

  test_mse = np.reshape(test.values, (test.values.shape[0]))
  LSTM_test_mse = np.reshape(LSTM_test.values, (LSTM_test.values.shape[0]))
  MSE_test = mean_squared_error(test_mse, LSTM_test_mse)

  return MSE_train, MSE_test, model

In [7]:
LOOK_BACK = 100
DROPUT = 0.3
NEURONS = 100
EPOCHS = 100
BATCH_SIZE = 32

model = LSTM_PAGS(LOOK_BACK, DROPUT, NEURONS, EPOCHS, BATCH_SIZE )



Epoch 1/100
23/23 [==============================] - 15s 326ms/step - loss: 0.1813 - mse: 0.1813
Epoch 2/100
23/23 [==============================] - 8s 326ms/step - loss: 0.0174 - mse: 0.0174
Epoch 3/100
23/23 [==============================] - 7s 323ms/step - loss: 0.0204 - mse: 0.0204
Epoch 4/100
23/23 [==============================] - 8s 328ms/step - loss: 0.0204 - mse: 0.0204
Epoch 5/100
23/23 [==============================] - 7s 326ms/step - loss: 0.0198 - mse: 0.0198
Epoch 6/100
23/23 [==============================] - 7s 322ms/step - loss: 0.0141 - mse: 0.0141
Epoch 7/100
23/23 [==============================] - 7s 325ms/step - loss: 0.0129 - mse: 0.0129
Epoch 8/100
23/23 [==============================] - 7s 325ms/step - loss: 0.0123 - mse: 0.0123
Epoch 9/100
23/23 [==============================] - 7s 325ms/step - loss: 0.0106 - mse: 0.0106
Epoch 10/100
23/23 [==============================] - 7s 323ms/step - loss: 0.0100 - mse: 0.0100
Epoch 11/100
23/23 [==================

In [8]:
model

(<tf.Tensor: shape=(), dtype=float32, numpy=1.8435901>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.9574647>,
 <keras.engine.sequential.Sequential at 0x7f99e2863550>)

In [9]:
model[2].save("model1.h5")

In [12]:
MODEL = load_model('model1.h5')

In [13]:
MODEL.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 100, 100)          40800     
                                                                 
 dropout_4 (Dropout)         (None, 100, 100)          0         
                                                                 
 lstm_5 (LSTM)               (None, 100, 100)          80400     
                                                                 
 dropout_5 (Dropout)         (None, 100, 100)          0         
                                                                 
 lstm_6 (LSTM)               (None, 100, 100)          80400     
                                                                 
 dropout_6 (Dropout)         (None, 100, 100)          0         
                                                                 
 lstm_7 (LSTM)               (None, 100)              

In [18]:
LSTM_PAGS_VAL(100, MODEL)

(<tf.Tensor: shape=(), dtype=float32, numpy=1.8435901>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.9574647>,
 <keras.engine.sequential.Sequential at 0x7f99dcbd2f90>)